In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import xgboost
from xgboost import XGBRegressor

In [2]:
region_dict = {
    "1": "BSk",
    "9": "Dfb",
    "4": "Cfa",
    "7": "Csb",
    "8": "Dfa",
    "3": "BWk",
    "10": "Dfc",
    "2": "BWh",
    "6": "Csa",
    "11": "Dsb",
    "0": "BSh",
    "5": "Cfb",
    "12": "Dsc",
    "13": "Dwa",
    "14": "Dwb"
}

In [3]:
features = ['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3', 
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__cfsv20',
 'wind-vwnd-250-2010-10',
 'contest-wind-h10-14d__wind-hgt-10',
 'wind-uwnd-925-2010-15',
 'wind-vwnd-250-2010-3',
 'nmme-prate-34w__cancm4',
 'sst-2010-4',
 'nmme0-prate-56w__ccsm30',
 'wind-uwnd-250-2010-16',
 'nmme0-prate-34w__gfdl0',
 'nmme0-prate-56w__cancm40',
 'sst-2010-1',
 'sst-2010-3',
 'wind-uwnd-250-2010-14',
 'nmme0-prate-34w__gfdlflora0',
 'nmme-prate-34w__gfdl',
 'wind-hgt-850-2010-9',
 'wind-vwnd-250-2010-1',
 'sst-2010-5',
 'cancm30',
 'nmme-prate-34w__ccsm4',
 'nmme0-prate-34w__nasa0',
 'wind-hgt-500-2010-9',
 'nmme0-prate-34w__cancm30',
 'wind-vwnd-250-2010-13',
 'wind_diff',
 'wind_diff_min',
 'wind_diff_min_month',
 'wind_diff_month',
 'diff_slp_first',
 'diff_wind_first',
 'diff_precip_first',
 'diff_sst_10_first',
 'diff_sst_10_min',
 'diff_sst_10_max',
 'range_sst_10',
 'scale_sst_10',
 'diff_sst_9_first',
 'diff_sst_9_min',
 'diff_sst_9_max',
 'range_sst_9',
 'scale_sst_9',
 'diff_sst_8_first',
 'diff_sst_8_min',
 'diff_sst_8_max',
 'range_sst_8',
 'scale_sst_8',
 'diff_sst_7_first',
 'diff_sst_7_min',
 'diff_sst_7_max',
 'range_sst_7',
 'scale_sst_7',
 'diff_sst_6_first',
 'diff_sst_6_min',
 'diff_sst_6_max',
 'range_sst_6',
 'scale_sst_6',
 'diff_sst_1_first',
 'diff_sst_1_min',
 'diff_sst_1_max',
 'range_sst_1',
 'scale_sst_1',
 'diff_sst_2_first',
 'diff_sst_2_min',
 'diff_sst_2_max',
 'range_sst_2',
 'scale_sst_2',
 'diff_sst_3_first',
 'diff_sst_3_min',
 'diff_sst_3_max',
 'range_sst_3',
 'scale_sst_3',
 'diff_sst_4_first',
 'diff_sst_4_min',
 'diff_sst_4_max',
 'range_sst_4',
 'scale_sst_4',
 'diff_sst_5_first',
 'diff_sst_5_min',
 'diff_sst_5_max',
 'range_sst_5',
 'scale_sst_5',
 'diff_pres_1_first',
 'diff_pres_1_min',
 'diff_pres_1_max',
 'range_pres_1',
 'scale_pres_1',
 'diff_ccsm3_month_1_first',
 'diff_ccsm3_month_1_min',
 'diff_ccsm3_month_1_max',
 'range_ccsm3_month_1',
 'scale_ccsm3_month_1',
 'diff_sst_1_month_1_first',
 'diff_sst_1_month_1_min',
 'diff_sst_1_month_1_max',
 'range_sst_1_month_1',
 'scale_sst_1_month_1',
 'sst_1_lag_1',
 'sst_5_lag_1',
 'sst_4_lag_1',
 'sst_1_lag_1_season',
 'sst_5_lag_1_season',
 'sst_4_lag_1_season']

target=["contest-tmp2m-14d__tmp2m"]

slp_col = 'contest-slp-14d__slp'
ccsm3_col = 'nmme-tmp2m-34w__ccsm3'
pres_col = 'contest-pres-sfc-gauss-14d__pres'
precip_col = 'contest-precip-14d__precip'
poten_col = 'contest-pevpr-sfc-gauss-14d__pevpr'
rhum_col = 'contest-rhum-sig995-14d__rhum'
elevation_col = 'elevation__elevation'
wind_col_500 = 'contest-wind-h500-14d__wind-hgt-500'
sst_1_col = 'sst-2010-1'
sst_5_col = 'sst-2010-5'
sst_3_col = 'sst-2010-3'
sst_4_col = 'sst-2010-4'
sst_6_col = 'sst-2010-6'
sst_7_col = 'sst-2010-7'
sst_8_col = 'sst-2010-8'
sst_9_col = 'sst-2010-9'
sst_10_col = 'sst-2010-10'
group_cols = ['startdate']

In [4]:
train_df = pd.read_csv('./train_h2o.csv')
test_df = pd.read_csv('./test_h2o.csv')

In [5]:
train_df['index'] = np.nan

In [6]:
sst_features_5 = []
for i in [1, 5, 3, 4]:
    sst_features_5 = sst_features_5 + [f'sst_{i}_neigh_5_mean', f'sst_{i}_neigh_5_min', f'sst_{i}_neigh_5_max']

sst_features_5_diag_pos = []
for i in [1, 5, 3, 4]:
    sst_features_5_diag_pos = sst_features_5_diag_pos + [f'sst_{i}_neigh_5_diag_pos_mean', f'sst_{i}_neigh_5_diag_pos_min', f'sst_{i}_neigh_5_diag_pos_max']


sst_features_time_5 = []
for i in [1, 5, 3, 4]:
    sst_features_time_5 = sst_features_time_5 + [f'sst_{i}_time_5_mean', f'sst_{i}_time_5_min', f'sst_{i}_time_5_max']

sst_features_time_7 = []
for i in [1, 5, 3, 4]:
    sst_features_time_7 = sst_features_time_7 + [f'sst_{i}_time_7_mean', f'sst_{i}_time_7_min', f'sst_{i}_time_7_max']


sst_features_time_14 = []
for i in [1, 5, 3, 4]:
    sst_features_time_14 = sst_features_time_14 + [f'sst_{i}_time_14_mean', f'sst_{i}_time_14_min', f'sst_{i}_time_14_max']

sst_features_time_9 = []
for i in [1, 5, 3, 4]:
    sst_features_time_9 = sst_features_time_9 + [f'sst_{i}_time_9_mean', f'sst_{i}_time_9_min', f'sst_{i}_time_9_max']


sst_features_time_11 = []
for i in [1, 5, 3, 4]:
    sst_features_time_11 = sst_features_time_11 + [f'sst_{i}_time_11_mean', f'sst_{i}_time_11_min', f'sst_{i}_time_11_max']

sst_features_11 = []
for i in [1, 5, 3, 4]:
    sst_features_11 = sst_features_11 + [f'sst_{i}_neigh_11_mean', f'sst_{i}_neigh_11_min', f'sst_{i}_neigh_11_max']

sst_features_3 = []
for i in [1, 5, 3, 4]:
    sst_features_3 = sst_features_3 + [f'sst_{i}_neigh_3_mean', f'sst_{i}_neigh_3_min', f'sst_{i}_neigh_3_max']

sst_features_7 = []
for i in [1, 5, 3, 4]:
    sst_features_7 = sst_features_7 + [f'sst_{i}_neigh_7_mean', f'sst_{i}_neigh_7_min', f'sst_{i}_neigh_7_max']

sst_features_5_lat = []
for i in [1, 5, 3, 4]:
    sst_features_5_lat = sst_features_5_lat + [f'sst_{i}_neigh_5_lat_mean', f'sst_{i}_neigh_5_lat_min', f'sst_{i}_neigh_5_lat_max']

sst_features_21_lat = []
for i in [1, 5, 3, 4]:
    sst_features_21_lat = sst_features_21_lat + [f'sst_{i}_neigh_21_lat_mean', f'sst_{i}_neigh_21_lat_min', f'sst_{i}_neigh_21_lat_max']

    
sst_features_5_lon = []
for i in [1, 5, 3, 4]:
    sst_features_5_lon = sst_features_5_lon + [f'sst_{i}_neigh_5_lon_mean', f'sst_{i}_neigh_5_lon_min', f'sst_{i}_neigh_5_lon_max']

sst_features_21_lat = []
for i in [1, 5, 3, 4]:
    sst_features_21_lat = sst_features_21_lat + [f'sst_{i}_neigh_21_lat_mean', f'sst_{i}_neigh_21_lat_min', f'sst_{i}_neigh_21_lat_max']
    
sst_features_21_lon = []
for i in [1, 5, 3, 4]:
    sst_features_21_lon = sst_features_21_lon + [f'sst_{i}_neigh_21_lon_mean', f'sst_{i}_neigh_21_lon_min', f'sst_{i}_neigh_21_lon_max']

    
sst_features_9_lon = []
for i in [1, 5, 3, 4]:
    sst_features_9_lon = sst_features_9_lon + [f'sst_{i}_neigh_9_lon_mean', f'sst_{i}_neigh_9_lon_min', f'sst_{i}_neigh_9_lon_max']

sst_features_11_lon = []
for i in [1, 5, 3, 4]:
    sst_features_11_lon = sst_features_11_lon + [f'sst_{i}_neigh_11_lon_mean', f'sst_{i}_neigh_11_lon_min', f'sst_{i}_neigh_11_lon_max']

sst_features_15_lon = []
for i in [1, 5, 3, 4]:
    sst_features_15_lon = sst_features_15_lon + [f'sst_{i}_neigh_15_lon_mean', f'sst_{i}_neigh_15_lon_min', f'sst_{i}_neigh_15_lon_max']

sst_features_11_lat = []
for i in [1, 5, 3, 4]:
    sst_features_11_lat = sst_features_11_lat + [f'sst_{i}_neigh_11_lat_mean', f'sst_{i}_neigh_11_lat_min', f'sst_{i}_neigh_11_lat_max']

sst_features_9_lat = []
for i in [1, 5, 3, 4]:
    sst_features_9_lat = sst_features_9_lat + [f'sst_{i}_neigh_9_lat_mean', f'sst_{i}_neigh_9_lat_min', f'sst_{i}_neigh_9_lat_max']
    
sst_features_9 = []
for i in [1, 5, 3, 4]:
    sst_features_9 = sst_features_9 + [f'sst_{i}_neigh_9_mean', f'sst_{i}_neigh_9_min', f'sst_{i}_neigh_9_max']

sst_features_13 = []
for i in [1, 5, 3, 4]:
    sst_features_13 = sst_features_13 + [f'sst_{i}_neigh_13_mean', f'sst_{i}_neigh_13_min', f'sst_{i}_neigh_13_max']


poten_feature_5 = ['poten_neigh_5_mean', 'poten_neigh_5_min', 'poten_neigh_5_max']
poten_feature_time_5 = ['poten_time_5_mean', 'poten_time_5_min', 'poten_time_5_max']
poten_feature_time_7 = ['poten_time_7_mean', 'poten_time_7_min', 'poten_time_7_max']
poten_feature_time_11 = ['poten_time_11_mean', 'poten_time_11_min', 'poten_time_11_max']
poten_feature_time_9 = ['poten_time_9_mean', 'poten_time_9_min', 'poten_time_9_max']

poten_feature_3 = ['poten_neigh_3_mean', 'poten_neigh_3_min', 'poten_neigh_3_max']
poten_feature_7 = ['poten_neigh_7_mean', 'poten_neigh_7_min', 'poten_neigh_7_max']
poten_feature_9 = ['poten_neigh_9_mean', 'poten_neigh_9_min', 'poten_neigh_9_max']
poten_feature_11 = ['poten_neigh_11_mean', 'poten_neigh_11_min', 'poten_neigh_11_max']
poten_feature_13 = ['poten_neigh_13_mean', 'poten_neigh_13_min', 'poten_neigh_13_max']



rhum_feature_5 = ['rhum_neigh_5_mean', 'rhum_neigh_5_min', 'rhum_neigh_5_max']
rhum_feature_5_diag_pos = ['rhum_neigh_5_diag_pos_mean', 'rhum_neigh_5_diag_pos_min', 'rhum_neigh_5_diag_pos_max']

rhum_feature_5_lat = ['rhum_neigh_5_lat_mean', 'rhum_neigh_5_lat_min', 'rhum_neigh_5_lat_max']
rhum_feature_5_lon = ['rhum_neigh_5_lon_mean', 'rhum_neigh_5_lon_min', 'rhum_neigh_5_lon_max']
rhum_feature_21_lon = ['rhum_neigh_21_lon_mean', 'rhum_neigh_21_lon_min', 'rhum_neigh_21_lon_max']
rhum_feature_21_lat = ['rhum_neigh_21_lat_mean', 'rhum_neigh_21_lat_min', 'rhum_neigh_21_lat_max']
rhum_feature_9_lon = ['rhum_neigh_9_lon_mean', 'rhum_neigh_9_lon_min', 'rhum_neigh_9_lon_max']
rhum_feature_11_lon = ['rhum_neigh_11_lon_mean', 'rhum_neigh_11_lon_min', 'rhum_neigh_11_lon_max']
rhum_feature_15_lon = ['rhum_neigh_15_lon_mean', 'rhum_neigh_15_lon_min', 'rhum_neigh_15_lon_max']
rhum_feature_11_lat = ['rhum_neigh_11_lat_mean', 'rhum_neigh_11_lat_min', 'rhum_neigh_11_lat_max']
rhum_feature_9_lat = ['rhum_neigh_9_lat_mean', 'rhum_neigh_9_lat_min', 'rhum_neigh_9_lat_max']
rhum_feature_time_3 = ['rhum_time_3_mean', 'rhum_time_3_min', 'rhum_time_3_max']
rhum_feature_time_5 = ['rhum_time_5_mean', 'rhum_time_5_min', 'rhum_time_5_max']
rhum_feature_time_7 = ['rhum_time_7_mean', 'rhum_time_7_min', 'rhum_time_7_max']
rhum_feature_time_11 = ['rhum_time_11_mean', 'rhum_time_11_min', 'rhum_time_11_max']
rhum_feature_time_7_mid = ['rhum_time_7_mid_mean', 'rhum_time_7_mid_min', 'rhum_time_7_mid_max']
rhum_feature_time_14 = ['rhum_time_14_mean', 'rhum_time_14_min', 'rhum_time_14_max']
rhum_feature_time_21 = ['rhum_time_21_mean', 'rhum_time_21_min', 'rhum_time_21_max']
rhum_feature_time_28 = ['rhum_time_28_mean', 'rhum_time_28_min', 'rhum_time_28_max']
rhum_feature_time_9 = ['rhum_time_9_mean', 'rhum_time_9_min', 'rhum_time_9_max']

rhum_feature_3 = ['rhum_neigh_3_mean', 'rhum_neigh_3_min', 'rhum_neigh_3_max']
rhum_feature_7 = ['rhum_neigh_7_mean', 'rhum_neigh_7_min', 'rhum_neigh_7_max']
rhum_feature_9 = ['rhum_neigh_9_mean', 'rhum_neigh_9_min', 'rhum_neigh_9_max']
rhum_feature_11 = ['rhum_neigh_11_mean', 'rhum_neigh_11_min', 'rhum_neigh_11_max']
rhum_feature_13 = ['rhum_neigh_13_mean', 'rhum_neigh_13_min', 'rhum_neigh_13_max']


slp_feature_5 = ['slp_neigh_5_mean', 'slp_neigh_5_min', 'slp_neigh_5_max']
slp_feature_5_diag_pos = ['slp_neigh_5_diag_pos_mean', 'slp_neigh_5_diag_pos_min', 'slp_neigh_5_diag_pos_max']
slp_feature_5_lat = ['slp_neigh_5_lat_mean', 'slp_neigh_5_lat_min', 'slp_neigh_5_lat_max']
slp_feature_5_lon = ['slp_neigh_5_lon_mean', 'slp_neigh_5_lon_min', 'slp_neigh_5_lon_max']
slp_feature_21_lon = ['slp_neigh_21_lon_mean', 'slp_neigh_21_lon_min', 'slp_neigh_21_lon_max']
slp_feature_21_lat = ['slp_neigh_21_lat_mean', 'slp_neigh_21_lat_min', 'slp_neigh_21_lat_max']
slp_feature_9_lon = ['slp_neigh_9_lon_mean', 'slp_neigh_9_lon_min', 'slp_neigh_9_lon_max']
slp_feature_11_lon = ['slp_neigh_11_lon_mean', 'slp_neigh_11_lon_min', 'slp_neigh_11_lon_max']
slp_feature_15_lon = ['slp_neigh_15_lon_mean', 'slp_neigh_15_lon_min', 'slp_neigh_15_lon_max']
slp_feature_11_lat = ['slp_neigh_11_lat_mean', 'slp_neigh_11_lat_min', 'slp_neigh_11_lat_max']
slp_feature_9_lat = ['slp_neigh_9_lat_mean', 'slp_neigh_9_lat_min', 'slp_neigh_9_lat_max']
slp_feature_time_5 = ['slp_time_5_mean', 'slp_time_5_min', 'slp_time_5_max']
slp_feature_time_7 = ['slp_time_7_mean', 'slp_time_7_min', 'slp_time_7_max']
slp_feature_time_11 = ['slp_time_11_mean', 'slp_time_11_min', 'slp_time_11_max']
slp_feature_time_14 = ['slp_time_14_mean', 'slp_time_14_min', 'slp_time_14_max']
slp_feature_time_9 = ['slp_time_9_mean', 'slp_time_9_min', 'slp_time_9_max']

slp_feature_3 = ['slp_neigh_3_mean', 'slp_neigh_3_min', 'slp_neigh_3_max']
slp_feature_7 = ['slp_neigh_7_mean', 'slp_neigh_7_min', 'slp_neigh_7_max']
slp_feature_9 = ['slp_neigh_9_mean', 'slp_neigh_9_min', 'slp_neigh_9_max']
slp_feature_11 = ['slp_neigh_11_mean', 'slp_neigh_11_min', 'slp_neigh_11_max']
slp_feature_13 = ['slp_neigh_13_mean', 'slp_neigh_13_min', 'slp_neigh_13_max']


precip_feature_5 = ['precip_neigh_5_mean', 'precip_neigh_5_min', 'precip_neigh_5_max']
precip_feature_5_diag_pos = ['precip_neigh_5_diag_pos_mean', 'precip_neigh_5_diag_pos_min', 'precip_neigh_5_diag_pos_max']
precip_feature_5_lat = ['precip_neigh_5_lat_mean', 'precip_neigh_5_lat_min', 'precip_neigh_5_lat_max']
precip_feature_5_lon = ['precip_neigh_5_lon_mean', 'precip_neigh_5_lon_min', 'precip_neigh_5_lon_max']
precip_feature_21_lon = ['precip_neigh_21_lon_mean', 'precip_neigh_21_lon_min', 'precip_neigh_21_lon_max']
precip_feature_21_lat = ['precip_neigh_21_lat_mean', 'precip_neigh_21_lat_min', 'precip_neigh_21_lat_max']
precip_feature_9_lon = ['precip_neigh_9_lon_mean', 'precip_neigh_9_lon_min', 'precip_neigh_9_lon_max']
precip_feature_11_lon = ['precip_neigh_11_lon_mean', 'precip_neigh_11_lon_min', 'precip_neigh_11_lon_max']
precip_feature_15_lon = ['precip_neigh_15_lon_mean', 'precip_neigh_15_lon_min', 'precip_neigh_15_lon_max']
precip_feature_11_lat = ['precip_neigh_11_lat_mean', 'precip_neigh_11_lat_min', 'precip_neigh_11_lat_max']
precip_feature_9_lat = ['precip_neigh_9_lat_mean', 'precip_neigh_9_lat_min', 'precip_neigh_9_lat_max']
precip_feature_time_3 = ['precip_time_3_mean', 'precip_time_3_min', 'precip_time_3_max']
precip_feature_time_5 = ['precip_time_5_mean', 'precip_time_5_min', 'precip_time_5_max']
precip_feature_time_7 = ['precip_time_7_mean', 'precip_time_7_min', 'precip_time_7_max']
precip_feature_time_11 = ['precip_time_11_mean', 'precip_time_11_min', 'precip_time_11_max']
precip_feature_time_7_mid = ['precip_time_7_mid_mean', 'precip_time_7_mid_min', 'precip_time_7_mid_max']
precip_feature_time_9 = ['precip_time_9_mean', 'precip_time_9_min', 'precip_time_9_max']
precip_feature_time_14 = ['precip_time_14_mean', 'precip_time_14_min', 'precip_time_14_max']
precip_feature_time_21 = ['precip_time_21_mean', 'precip_time_21_min', 'precip_time_21_max']
precip_feature_time_28 = ['precip_time_28_mean', 'precip_time_28_min', 'precip_time_28_max']

precip_feature_3 = ['precip_neigh_3_mean', 'precip_neigh_3_min', 'precip_neigh_3_max']
precip_feature_7 = ['precip_neigh_7_mean', 'precip_neigh_7_min', 'precip_neigh_7_max']
precip_feature_9 = ['precip_neigh_9_mean', 'precip_neigh_9_min', 'precip_neigh_9_max']
precip_feature_11 = ['precip_neigh_11_mean', 'precip_neigh_11_min', 'precip_neigh_11_max']
precip_feature_13 = ['precip_neigh_13_mean', 'precip_neigh_13_min', 'precip_neigh_13_max']


pres_feature_5 = ['pres_neigh_5_mean', 'pres_neigh_5_min', 'pres_neigh_5_max']
pres_feature_5_diag_pos = ['pres_neigh_5_diag_pos_mean', 'pres_neigh_5_diag_pos_min', 'pres_neigh_5_diag_pos_max']
pres_feature_5_lat = ['pres_neigh_5_lat_mean', 'pres_neigh_5_lat_min', 'pres_neigh_5_lat_max']
pres_feature_5_lon = ['pres_neigh_5_lon_mean', 'pres_neigh_5_lon_min', 'pres_neigh_5_lon_max']
pres_feature_21_lon = ['pres_neigh_21_lon_mean', 'pres_neigh_21_lon_min', 'pres_neigh_21_lon_max']
pres_feature_21_lat = ['pres_neigh_21_lat_mean', 'pres_neigh_21_lat_min', 'pres_neigh_21_lat_max']
pres_feature_9_lon = ['pres_neigh_9_lon_mean', 'pres_neigh_9_lon_min', 'pres_neigh_9_lon_max']
pres_feature_15_lon = ['pres_neigh_15_lon_mean', 'pres_neigh_15_lon_min', 'pres_neigh_15_lon_max']
pres_feature_11_lat = ['pres_neigh_11_lat_mean', 'pres_neigh_11_lat_min', 'pres_neigh_11_lat_max']
pres_feature_9_lat = ['pres_neigh_9_lat_mean', 'pres_neigh_9_lat_min', 'pres_neigh_9_lat_max']
pres_feature_time_3 = ['pres_time_3_mean', 'pres_time_3_min', 'pres_time_3_max']
pres_feature_time_5 = ['pres_time_5_mean', 'pres_time_5_min', 'pres_time_5_max']
pres_feature_time_7 = ['pres_time_7_mean', 'pres_time_7_min', 'pres_time_7_max']
pres_feature_time_11 = ['pres_time_11_mean', 'pres_time_11_min', 'pres_time_11_max']
pres_feature_time_7_mid = ['pres_time_7_mid_mean', 'pres_time_7_mid_min', 'pres_time_7_mid_max']
pres_feature_time_9 = ['pres_time_9_mean', 'pres_time_9_min', 'pres_time_9_max']
pres_feature_time_14 = ['pres_time_14_mean', 'pres_time_14_min', 'pres_time_14_max']
pres_feature_time_21 = ['pres_time_21_mean', 'pres_time_21_min', 'pres_time_21_max']
pres_feature_time_28 = ['pres_time_28_mean', 'pres_time_28_min', 'pres_time_28_max']

pres_feature_3 = ['pres_neigh_3_mean', 'pres_neigh_3_min', 'pres_neigh_3_max']
pres_feature_7 = ['pres_neigh_7_mean', 'pres_neigh_7_min', 'pres_neigh_7_max']
pres_feature_9 = ['pres_neigh_9_mean', 'pres_neigh_9_min', 'pres_neigh_9_max']
pres_feature_11 = ['pres_neigh_11_mean', 'pres_neigh_11_min', 'pres_neigh_11_max']
pres_feature_13 = ['pres_neigh_13_mean', 'pres_neigh_13_min', 'pres_neigh_13_max']



wind_feature_11 = ['wind_500_neigh_11_mean', 'wind_500_neigh_11_min', 'wind_500_neigh_11_max']
wind_feature_9 = ['wind_500_neigh_9_mean', 'wind_500_neigh_9_min', 'wind_500_neigh_9_max']
wind_feature_5 = ['wind_500_neigh_5_mean', 'wind_500_neigh_5_min', 'wind_500_neigh_5_max']
wind_feature_5_diag_pos = ['wind_500_neigh_5_diag_pos_mean', 'wind_500_neigh_5_diag_pos_min', 'wind_500_neigh_5_diag_pos_max']
wind_feature_5_lat = ['wind_500_neigh_5_lat_mean', 'wind_500_neigh_5_lat_min', 'wind_500_neigh_5_lat_max']
wind_feature_5_lon = ['wind_500_neigh_5_lon_mean', 'wind_500_neigh_5_lon_min', 'wind_500_neigh_5_lon_max']
wind_feature_21_lon = ['wind_500_neigh_21_lon_mean', 'wind_500_neigh_21_lon_min', 'wind_500_neigh_21_lon_max']
wind_feature_21_lat = ['wind_500_neigh_21_lat_mean', 'wind_500_neigh_21_lat_min', 'wind_500_neigh_21_lat_max']
wind_feature_9_lon = ['wind_500_neigh_9_lon_mean', 'wind_500_neigh_9_lon_min', 'wind_500_neigh_9_lon_max']
wind_feature_11_lon = ['wind_500_neigh_11_lon_mean', 'wind_500_neigh_11_lon_min', 'wind_500_neigh_11_lon_max']
wind_feature_15_lon = ['wind_500_neigh_15_lon_mean', 'wind_500_neigh_15_lon_min', 'wind_500_neigh_15_lon_max']
wind_feature_11_lat = ['wind_500_neigh_11_lat_mean', 'wind_500_neigh_11_lat_min', 'wind_500_neigh_11_lat_max']
wind_feature_9_lat = ['wind_500_neigh_9_lat_mean', 'wind_500_neigh_9_lat_min', 'wind_500_neigh_9_lat_max']
wind_feature_time_5 = ['wind_500_time_5_mean', 'wind_500_time_5_min', 'wind_500_time_5_max']
wind_feature_time_7 = ['wind_500_time_7_mean', 'wind_500_time_7_min', 'wind_500_time_7_max']
wind_feature_time_14 = ['wind_500_time_14_mean', 'wind_500_time_14_min', 'wind_500_time_14_max']
wind_feature_time_9 = ['wind_500_time_9_mean', 'wind_500_time_9_min', 'wind_500_time_9_max']
wind_feature_time_11 = ['wind_500_time_11_mean', 'wind_500_time_11_min', 'wind_500_time_11_max']
wind_feature_7 = ['wind_500_neigh_7_mean', 'wind_500_neigh_7_min', 'wind_500_neigh_7_max']
wind_feature_3 = ['wind_500_neigh_3_mean', 'wind_500_neigh_3_min', 'wind_500_neigh_3_max']
wind_feature_13 = ['wind_500_neigh_13_mean', 'wind_500_neigh_13_min', 'wind_500_neigh_13_max']

In [8]:
selected_cols = features + sst_features_5 + wind_feature_5 + slp_feature_5 + precip_feature_5 + pres_feature_5 + rhum_feature_5 + slp_feature_3 + precip_feature_3 + pres_feature_3  + slp_feature_7 + rhum_feature_7 + sst_features_7 + precip_feature_7 + pres_feature_7 + wind_feature_7 + slp_feature_9 + wind_feature_9 + pres_feature_9 + precip_feature_9 + rhum_feature_9 + rhum_feature_13 + \
rhum_feature_time_7 + pres_feature_time_7 + precip_feature_time_7 + \
rhum_feature_time_14 + pres_feature_time_14 + precip_feature_time_14 + \
sst_features_5_lat + wind_feature_5_lat + slp_feature_5_lat + precip_feature_5_lat + pres_feature_5_lat + rhum_feature_5_lat + \
sst_features_5_lon + wind_feature_5_lon + slp_feature_5_lon + precip_feature_5_lon + pres_feature_5_lon + rhum_feature_5_lon + \
slp_feature_5_diag_pos + precip_feature_5_diag_pos + pres_feature_5_diag_pos + rhum_feature_5_diag_pos

In [9]:
cols = selected_cols
len(cols)

302

In [10]:
# out_cols = ["sst_4_neigh_7_max", "sst_5_neigh_5_lat_min", "sst_1_neigh_5_lon_min", "sst_3_neigh_5_lon_max", "sst_4_neigh_5_lat_max", "sst_5_neigh_5_min", "sst_1_neigh_7_max", "sst_5_neigh_7_max", "sst_1_neigh_5_lat_max", "sst_3_neigh_7_max", "sst_3_neigh_7_min", "sst_4_neigh_7_min", "sst_5_neigh_5_lon_max", "sst_1_neigh_7_min", "sst_5_neigh_7_min", "sst_5_neigh_5_max", "sst_4_neigh_5_lat_min", "sst_1_neigh_5_lat_min", "sst_5_neigh_5_lat_max", "sst_4_neigh_5_lon_min", "sst_1_neigh_5_max", "sst_3_neigh_5_lat_max", "sst_5_neigh_5_lon_min", "sst_4_neigh_5_max", "sst_4_neigh_5_lon_max", "sst_3_neigh_5_max", "sst_1_neigh_5_lon_max", "sst_3_neigh_5_lon_min", "sst_1_neigh_5_min", "sst_3_neigh_5_lat_min", "sst_4_neigh_5_min", "sst_3_neigh_5_min"]
# cols = [c for c in cols if c not in out_cols]

In [11]:

def train_test_by_df(train_df_, test_df_, iterations):
  X = train_df_[cols].values
  y = train_df_[target[0]].values

  skf = KFold(n_splits=5, random_state=2048, shuffle=True)
  skf.get_n_splits(X, y)

  train_index_list = []
  test_index_list = []
  train_x_list = []
  val_x_list = []
  train_y_list = []
  val_y_list = []

  for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index]) 
  
    
  cat_models = []
  for i in tqdm(range(5)):
    train_x, val_x = train_x_list[i], val_x_list[i]
    train_y, val_y = train_y_list[i], val_y_list[i]
    clf = CatBoostRegressor( iterations = iterations, verbose = 1000, task_type = 'GPU', devices = '1')
    clf.fit(train_x, train_y, eval_set=[(val_x, val_y)])
    cat_models.append(clf)

  preds = []
  for cat_model in tqdm(cat_models):
      preds.append(cat_model.predict(test_df_[cols].values))
  s = 0
  for pred in preds:
      s += pred
  s = s/5
  
  return s, cat_models


In [ ]:
s, cat_models = train_test_by_df(train_df, test_df, 1000000)

5it [00:03,  1.25it/s]
  0%|                                                                                                | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.01606
0:	learn: 9.7269030	test: 9.7596316	best: 9.7596316 (0)	total: 6.09ms	remaining: 1h 41m 32s
1000:	learn: 1.0259589	test: 1.0336681	best: 1.0336681 (1000)	total: 5.79s	remaining: 1h 36m 23s
2000:	learn: 0.7523876	test: 0.7639905	best: 0.7639905 (2000)	total: 11.6s	remaining: 1h 36m 10s
3000:	learn: 0.6296833	test: 0.6429484	best: 0.6429484 (3000)	total: 17.4s	remaining: 1h 36m 15s
4000:	learn: 0.5556118	test: 0.5703827	best: 0.5703827 (4000)	total: 23.2s	remaining: 1h 36m 6s
5000:	learn: 0.5043459	test: 0.5203592	best: 0.5203592 (5000)	total: 28.9s	remaining: 1h 35m 59s
6000:	learn: 0.4662788	test: 0.4833869	best: 0.4833869 (6000)	total: 34.7s	remaining: 1h 35m 49s
7000:	learn: 0.4358579	test: 0.4540165	best: 0.4540165 (7000)	total: 40.8s	remaining: 1h 36m 26s
8000:	learn: 0.4111930	test: 0.4304944	best: 0.4304944 (8000)	total: 49.8s	remaining: 1h 42m 52s
9000:	learn: 0.3902242	test: 0.4105700	best: 0.4105700 (9000)	total: 56.3s	remaining: 1h 43m 15s
10000:	

83000:	learn: 0.1259318	test: 0.1768016	best: 0.1768016 (83000)	total: 8m 12s	remaining: 1h 30m 39s
84000:	learn: 0.1250523	test: 0.1761442	best: 0.1761442 (84000)	total: 8m 18s	remaining: 1h 30m 31s
85000:	learn: 0.1241788	test: 0.1754941	best: 0.1754941 (85000)	total: 8m 23s	remaining: 1h 30m 23s
86000:	learn: 0.1233388	test: 0.1748684	best: 0.1748684 (86000)	total: 8m 29s	remaining: 1h 30m 14s
87000:	learn: 0.1225085	test: 0.1742554	best: 0.1742554 (87000)	total: 8m 35s	remaining: 1h 30m 5s
88000:	learn: 0.1216875	test: 0.1736388	best: 0.1736388 (88000)	total: 8m 40s	remaining: 1h 29m 58s
89000:	learn: 0.1208809	test: 0.1730487	best: 0.1730487 (89000)	total: 8m 46s	remaining: 1h 29m 50s
90000:	learn: 0.1200878	test: 0.1724612	best: 0.1724612 (90000)	total: 8m 52s	remaining: 1h 29m 43s
91000:	learn: 0.1193058	test: 0.1718913	best: 0.1718913 (91000)	total: 8m 58s	remaining: 1h 29m 37s
92000:	learn: 0.1185384	test: 0.1713254	best: 0.1713254 (92000)	total: 9m 4s	remaining: 1h 29m 32s
93

164000:	learn: 0.0828776	test: 0.1470882	best: 0.1470882 (164000)	total: 16m 4s	remaining: 1h 21m 57s
165000:	learn: 0.0825511	test: 0.1468836	best: 0.1468836 (165000)	total: 16m 10s	remaining: 1h 21m 50s
166000:	learn: 0.0822257	test: 0.1466813	best: 0.1466813 (166000)	total: 16m 16s	remaining: 1h 21m 44s
167000:	learn: 0.0818988	test: 0.1464839	best: 0.1464839 (167000)	total: 16m 21s	remaining: 1h 21m 37s
168000:	learn: 0.0815729	test: 0.1462810	best: 0.1462810 (168000)	total: 16m 27s	remaining: 1h 21m 31s
169000:	learn: 0.0812503	test: 0.1460871	best: 0.1460871 (169000)	total: 16m 33s	remaining: 1h 21m 24s
170000:	learn: 0.0809358	test: 0.1458867	best: 0.1458867 (170000)	total: 16m 39s	remaining: 1h 21m 18s
171000:	learn: 0.0806206	test: 0.1456964	best: 0.1456964 (171000)	total: 16m 45s	remaining: 1h 21m 12s
172000:	learn: 0.0803091	test: 0.1455154	best: 0.1455154 (171999)	total: 16m 50s	remaining: 1h 21m 6s
173000:	learn: 0.0799962	test: 0.1453304	best: 0.1453304 (173000)	total: 16

244000:	learn: 0.0629043	test: 0.1356001	best: 0.1356001 (244000)	total: 23m 51s	remaining: 1h 13m 55s
245000:	learn: 0.0627133	test: 0.1354961	best: 0.1354961 (245000)	total: 23m 57s	remaining: 1h 13m 49s
246000:	learn: 0.0625257	test: 0.1353986	best: 0.1353986 (246000)	total: 24m 3s	remaining: 1h 13m 42s
247000:	learn: 0.0623388	test: 0.1353000	best: 0.1353000 (247000)	total: 24m 8s	remaining: 1h 13m 37s
248000:	learn: 0.0621537	test: 0.1352047	best: 0.1352047 (248000)	total: 24m 14s	remaining: 1h 13m 31s
249000:	learn: 0.0619706	test: 0.1351092	best: 0.1351092 (249000)	total: 24m 20s	remaining: 1h 13m 25s
250000:	learn: 0.0617868	test: 0.1350140	best: 0.1350140 (250000)	total: 24m 26s	remaining: 1h 13m 18s
251000:	learn: 0.0616071	test: 0.1349195	best: 0.1349195 (250999)	total: 24m 32s	remaining: 1h 13m 12s
252000:	learn: 0.0614251	test: 0.1348244	best: 0.1348244 (252000)	total: 24m 37s	remaining: 1h 13m 6s
253000:	learn: 0.0612444	test: 0.1347315	best: 0.1347315 (252999)	total: 24m

...

889000:	learn: 0.0183302	test: 0.1188905	best: 0.1188905 (888999)	total: 1h 38m 27s	remaining: 12m 17s
890000:	learn: 0.0183042	test: 0.1188840	best: 0.1188840 (889997)	total: 1h 38m 33s	remaining: 12m 10s
891000:	learn: 0.0182784	test: 0.1188777	best: 0.1188777 (891000)	total: 1h 38m 39s	remaining: 12m 4s
892000:	learn: 0.0182529	test: 0.1188714	best: 0.1188714 (891996)	total: 1h 38m 45s	remaining: 11m 57s
893000:	learn: 0.0182269	test: 0.1188645	best: 0.1188645 (892994)	total: 1h 38m 51s	remaining: 11m 50s
894000:	learn: 0.0182006	test: 0.1188581	best: 0.1188581 (893995)	total: 1h 38m 57s	remaining: 11m 43s
895000:	learn: 0.0181741	test: 0.1188511	best: 0.1188511 (894999)	total: 1h 39m 2s	remaining: 11m 37s
896000:	learn: 0.0181484	test: 0.1188450	best: 0.1188450 (896000)	total: 1h 39m 8s	remaining: 11m 30s
897000:	learn: 0.0181226	test: 0.1188387	best: 0.1188387 (896998)	total: 1h 39m 14s	remaining: 11m 23s
898000:	learn: 0.0180963	test: 0.1188320	best: 0.1188320 (898000)	total: 1h 

970000:	learn: 0.0163653	test: 0.1184063	best: 0.1184063 (969995)	total: 1h 46m 29s	remaining: 3m 17s
971000:	learn: 0.0163429	test: 0.1184010	best: 0.1184010 (970998)	total: 1h 46m 35s	remaining: 3m 10s
972000:	learn: 0.0163207	test: 0.1183958	best: 0.1183958 (972000)	total: 1h 46m 41s	remaining: 3m 4s
973000:	learn: 0.0162985	test: 0.1183904	best: 0.1183904 (972994)	total: 1h 46m 46s	remaining: 2m 57s
974000:	learn: 0.0162758	test: 0.1183844	best: 0.1183844 (974000)	total: 1h 46m 52s	remaining: 2m 51s
975000:	learn: 0.0162538	test: 0.1183794	best: 0.1183794 (975000)	total: 1h 46m 58s	remaining: 2m 44s
976000:	learn: 0.0162310	test: 0.1183742	best: 0.1183742 (975993)	total: 1h 47m 4s	remaining: 2m 37s
977000:	learn: 0.0162092	test: 0.1183692	best: 0.1183692 (976999)	total: 1h 47m 10s	remaining: 2m 31s
978000:	learn: 0.0161866	test: 0.1183640	best: 0.1183640 (977998)	total: 1h 47m 16s	remaining: 2m 24s
979000:	learn: 0.0161641	test: 0.1183582	best: 0.1183582 (979000)	total: 1h 47m 21s	

 80%|█████████████████████████████████████████████████████████████████▌                | 4/5 [8:40:52<2:14:42, 8082.69s/it]

Learning rate set to 0.01606
0:	learn: 9.7397903	test: 9.7087480	best: 9.7087480 (0)	total: 6.37ms	remaining: 1h 46m 5s
1000:	learn: 1.0245614	test: 1.0292570	best: 1.0292570 (1000)	total: 5.85s	remaining: 1h 37m 18s
2000:	learn: 0.7523416	test: 0.7597775	best: 0.7597775 (2000)	total: 11.8s	remaining: 1h 37m 54s
3000:	learn: 0.6296184	test: 0.6400537	best: 0.6400537 (3000)	total: 17.7s	remaining: 1h 37m 59s
4000:	learn: 0.5557925	test: 0.5681458	best: 0.5681458 (4000)	total: 23.6s	remaining: 1h 37m 55s
5000:	learn: 0.5049241	test: 0.5189267	best: 0.5189267 (5000)	total: 29.6s	remaining: 1h 38m 18s
6000:	learn: 0.4665896	test: 0.4821721	best: 0.4821721 (6000)	total: 35.5s	remaining: 1h 37m 59s
7000:	learn: 0.4361641	test: 0.4530378	best: 0.4530378 (7000)	total: 41.4s	remaining: 1h 37m 45s
8000:	learn: 0.4113619	test: 0.4294864	best: 0.4294864 (8000)	total: 47.2s	remaining: 1h 37m 32s
9000:	learn: 0.3902970	test: 0.4096748	best: 0.4096748 (9000)	total: 53.2s	remaining: 1h 37m 32s
10000:	

83000:	learn: 0.1254662	test: 0.1758581	best: 0.1758581 (83000)	total: 8m 18s	remaining: 1h 31m 43s
84000:	learn: 0.1245906	test: 0.1751959	best: 0.1751959 (84000)	total: 8m 24s	remaining: 1h 31m 36s
85000:	learn: 0.1237406	test: 0.1745644	best: 0.1745644 (85000)	total: 8m 29s	remaining: 1h 31m 28s
86000:	learn: 0.1228994	test: 0.1739334	best: 0.1739334 (86000)	total: 8m 35s	remaining: 1h 31m 21s
87000:	learn: 0.1220747	test: 0.1733188	best: 0.1733188 (87000)	total: 8m 41s	remaining: 1h 31m 14s
88000:	learn: 0.1212633	test: 0.1727100	best: 0.1727100 (88000)	total: 8m 47s	remaining: 1h 31m 5s
89000:	learn: 0.1204573	test: 0.1721223	best: 0.1721223 (89000)	total: 8m 53s	remaining: 1h 30m 58s
90000:	learn: 0.1196737	test: 0.1715484	best: 0.1715484 (90000)	total: 8m 59s	remaining: 1h 30m 49s
91000:	learn: 0.1189072	test: 0.1709823	best: 0.1709823 (91000)	total: 9m 4s	remaining: 1h 30m 43s
92000:	learn: 0.1181485	test: 0.1704161	best: 0.1704161 (92000)	total: 9m 10s	remaining: 1h 30m 35s
93

In [ ]:
output_df = pd.DataFrame(columns = ['index', 'pred'])
output_df['index'] = [int(i) for i in test_df['index']]
output_df['pred'] = s
res_df = pd.read_csv('./sample_solution.csv')
res_df = res_df.merge(output_df, how='left', on = ['index'])
res_df['contest-tmp2m-14d__tmp2m'] = res_df['pred']
res_df[[target[0], 'index']].to_csv('submission_million_tree.csv', index=False)

In [ ]:
# best_df = pd.read_csv('./best_sub_cat.csv')
# t = best_df[target[0]] * ((1 + 0.1 / 30))
# best_df[target[0]] = t
# best_df.to_csv('submission_post.csv', index=False)

In [ ]:
# res_df = pd.read_csv('./sample_solution.csv')
# cat_df = pd.read_csv('./submission_fe_pos_v2.csv')
# lgb_df = pd.read_csv('./best_lgb.csv')
# xgb_df = pd.read_csv('./best_xgb.csv')
# s = cat_df[target[0]] * 0.73 + lgb_df[target[0]] * 0.25 + xgb_df[target[0]] * 0.02
# res_df[target[0]] = s
# res_df[[target[0], 'index']].to_csv('submission_ensemble.csv', index=False)